In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

SEED = 515
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Faster Sentiment Analysis
This notebook follows this tutorial: https://github.com/bentrevett/pytorch-sentiment-analysis.

In [2]:
def append_bigrams(x):
    bigrams = set(["%s %s" % (f, s) for f, s in zip(x[:-1], x[1:])])
    x.extend(list(bigrams))
    return x

append_bigrams("This film is the best".split())

['This',
 'film',
 'is',
 'the',
 'best',
 'the best',
 'This film',
 'is the',
 'film is']

In [3]:
import torchtext
from torchtext.data import Field, LabelField, BucketIterator

TEXT = Field(tokenize='spacy', preprocessing=append_bigrams, include_lengths=True)
LABEL = LabelField(dtype=torch.float)

train_data, test_data = torchtext.datasets.IMDB.splits(TEXT, LABEL, root='data')
train_data, valid_data = train_data.split()

In [4]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE, 
                 vectors="glove.6B.100d", vectors_cache="vector_cache", 
                 unk_init=torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [5]:
# The bi-grams are included in the `Dataset`.
print(train_data[0].text)
# The bi-grams are also included in the `TEXT.vocab`.
print(TEXT.vocab.itos[-10:])

['I', 'LOVE', 'this', 'show', ',', 'it', "'s", 'sure', 'to', 'be', 'a', 'winner', '.', 'Jessica', 'Alba', 'does', 'a', 'great', 'job', ',', 'it', "'s", 'about', 'time', 'we', 'have', 'a', 'kick', '-', 'ass', 'girl', 'who', "'s", 'not', 'the', 'cutesy', 'type', '.', 'The', 'entire', 'cast', 'is', 'wonderful', 'and', 'all', 'the', 'episopes', 'have', 'good', 'plots', '.', 'Everything', 'is', 'layed', 'out', 'well', ',', 'and', 'thought', 'over', '.', 'To', 'put', 'it', 'together', 'must', 'have', 'taken', 'a', 'while', ',', 'because', 'it', 'was', "n't", 'someone', 'in', 'a', 'hurry', 'that', 'just', 'slapped', 'something', 'together', '.', 'It', "'s", 'a', 'GREAT', 'show', 'altogether', '.', '. The', 'this show', 'it together', 'put it', 'must have', 'Jessica Alba', 'entire cast', 'it was', 'type .', 'a kick', "'s sure", '- ass', 'cast is', "was n't", 'together must', 'all the', 'layed out', 'not the', 'is wonderful', 'someone in', 'we have', 'in a', '. Everything', "'s a", 'plots .', '

In [6]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE, device=device)

In [8]:
for batch in train_iterator:
    break
text, text_lens = batch.text
print(text)
print(text_lens)

tensor([[   70,    11,    11,  ...,    70, 21380,    25],
        [   22,   805,    58,  ...,    22,     5,   483],
        [    9,     2,    28,  ...,     9,  4562,     7],
        ...,
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1],
        [    1,     1,     1,  ...,     1,     1,     1]])
tensor([ 324,  550,  583, 1099,  274,  330,  781,  405,  939,  351, 1617,  271,
         356,  267,  745,  640,  462,  223,  161, 1513,  700,  258,  268,  179,
         534,  383,  409, 1690,  506,  271,  258,  159,  282,  582,  343,  199,
         417,  863,  537,  336,  138,  714,  819,  330,  398,  511,  269,  402,
         303, 1435,  543,  341,  384,  360,  445, 2226,  828,  317,  272,  810,
         327,  277,  207,  882])


# Build Model

In [15]:
class Classifier(nn.Module):
    def __init__(self, in_dim, emb_dim, out_dim, pad_idx):
        super().__init__()
        self.emb = nn.Embedding(in_dim, emb_dim, padding_idx=pad_idx)
        self.fc = nn.Linear(emb_dim, out_dim)

    def forward(self, text, text_lens):
        # text: (step, batch)
        embedded = self.emb(text)
        
        # Pooling along steps
        # pooled: (batch, emb)
        pooled = embedded.sum(dim=0) / text_lens.unsqueeze(1)
        return self.fc(pooled)

In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

IN_DIM = len(TEXT.vocab)
EMB_DIM = 100
OUT_DIM = 1
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

classifer = Classifier(IN_DIM, EMB_DIM, OUT_DIM, PAD_IDX).to(device)
count_parameters(classifer)

2500301

In [17]:
# Initialize Embeddings with Pre-Trained Vectors
classifer.emb.weight.data.copy_(TEXT.vocab.vectors)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

classifer.emb.weight.data[UNK_IDX].zero_()
classifer.emb.weight.data[PAD_IDX].zero_()

print(classifer.emb.weight[:5, :8])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281, -0.3996,  0.0832,  0.0440, -0.3914,  0.3344],
        [-0.1077,  0.1105,  0.5981, -0.5436,  0.6740,  0.1066,  0.0389,  0.3548],
        [-0.3398,  0.2094,  0.4635, -0.6479, -0.3838,  0.0380,  0.1713,  0.1598]],
       grad_fn=<SliceBackward>)


# Train Model

In [18]:
optimizer = optim.Adam(classifer.parameters())
# Binary cross entropy with logits. 
# The binary version of cross entropy loss. 
loss_func = nn.BCEWithLogitsLoss().to(device)

In [19]:
def train_epoch(classifer, iterator, optimizer, loss_func):
    classifer.train()
    epoch_loss = 0
    epoch_acc = 0
    for batch in iterator:
        # Forward pass
        text, text_lens = batch.text
        preds = classifer(text, text_lens).squeeze(-1)
        # Calculate loss
        loss = loss_func(preds, batch.label)
        # Backward propagation
        optimizer.zero_grad()
        loss.backward()
        # Update weights
        optimizer.step()
        # Accumulate loss and acc
        epoch_loss += loss.item()
        epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

def eval_epoch(classifer, iterator, loss_func):
    classifer.eval()
    epoch_loss = 0
    epoch_acc = 0
    with torch.no_grad():
        for batch in iterator:
            # Forward pass
            text, text_lens = batch.text
            preds = classifer(text, text_lens).squeeze(-1)
            # Calculate loss
            loss = loss_func(preds, batch.label)
            # Accumulate loss and acc
            epoch_loss += loss.item()
            epoch_acc += (torch.round(torch.sigmoid(preds)) == batch.label).sum().item() / preds.size(0)
    return epoch_loss/len(iterator), epoch_acc/len(iterator)

In [20]:
train_epoch(classifer, train_iterator, optimizer, loss_func)

NameError: name 'emb' is not defined